# 1. HuggingFaceH4/CodeAlpaca_20K Python split

In [ ]:
import pyarrow.parquet as pq
import pyarrow as pa
from datasets import Dataset, DatasetDict, concatenate_datasets
from datasets import Features, Value
from datasets import  load_dataset, DownloadConfig
from datasets import load_from_disk
import pandas as pd

In [2]:
h4ca20k = load_dataset("HuggingFaceH4/CodeAlpaca_20K")

d:\Disk_D\programming_software\AI\Anaconda3\envs\pettingZoo_Langchain\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [3]:
print(h4ca20k)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 18019
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2003
    })
})


In [11]:
print(h4ca20k["train"])

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 18019
})


In [61]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=open('./api/apikey/glmkeys.txt', 'r').read().strip()) 

In [23]:
def is_py(prompts):
    py_dataset = Dataset.from_dict({})
    for i in prompts:

        response = client.chat.completions.create(
            model="glm-4", 
    
            messages=[


                {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
                {"role": "user", "content": i["prompt"]},
                
        
            ],
        )

        if(response.choices[0].message.content == "1"):
            py_dataset.add_item(i)

    return py_dataset


In [69]:
prompts  = h4ca20k["train"]

In [70]:

features = Features({
    'prompt': Value('string'),
    'completion': Value('string')
})


py_dataset = Dataset.from_dict({'prompt':[], 'completion':[]},features=features)

In [72]:

cot = 0
for i in prompts:

    response = client.chat.completions.create(
        model="glm-4", 
    
        messages=[


            {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
            {"role": "user", "content": i["prompt"]},
                
        
        ],
    )

    if (response.choices[0].message.content == "1"):
        
        
        if not(cot % 100):
            cot = cot+1
            print(cot)
            continue
        cot = cot+1

        py_dataset = concatenate_datasets([py_dataset,Dataset.from_dict({'prompt':[i['prompt']],'completion':[i['completion']]})])
    
    

1
101
201
301
401
501
601
701
801
901
1001
1101
1201
1301
1401
1501
1601
1701
1801
1901
2001
2101
2201
2301
2401
2501
2601
2701
2801
2901
3001
3101
3201
3301
3401
3501
3601
3701
3801
3901
4001
4101
4201
4301
4401
4501
4601
4701


In [73]:
print(py_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 4777
})


In [62]:
print(py_dataset)

Dataset({
    features: ['prompt', 'completion'],
    num_rows: 548
})


In [63]:
py_dataset.save_to_disk('./data/codeNLU/h4ca20k_py/h4ca20k_py_test.parquet')

Saving the dataset (1/1 shards): 100%|██████████| 548/548 [00:00<00:00, 42153.81 examples/s]


In [58]:
py_test = py_dataset

In [74]:
py_train = py_dataset

In [75]:
py_train.save_to_disk('./data/codeNLU/h4ca20k_py/h4ca20k_py_train.parquet')

Saving the dataset (1/1 shards): 100%|██████████| 4777/4777 [00:00<00:00, 135963.94 examples/s]


In [68]:
h4ca20k_pytest = load_from_disk("./data/codeNLU/h4ca20k_py/h4ca20k_py_test.parquet")

pytest_table = pa.Table.from_pandas(h4ca20k_pytest.to_pandas())

pq.write_table(pytest_table, './data/codeNLU/h4ca20k_py/h4ca20k_py_test_.parquet')



In [76]:
h4ca20k_pytrain = load_from_disk("./data/codeNLU/h4ca20k_py/h4ca20k_py_train.parquet")

pytrain_table = pa.Table.from_pandas(h4ca20k_pytrain.to_pandas())

pq.write_table(pytrain_table, './data/codeNLU/h4ca20k_py/h4ca20k_py_train_.parquet')


In [79]:
dataset_dict = DatasetDict({
    'train': py_train,
    'test': py_test
})


dataset_dict.save_to_disk("./data/codeNLU/h4ca20k_py/h4ca20k_py.parquet")


Saving the dataset (1/1 shards): 100%|██████████| 548/548 [00:00<00:00, 90953.21 examples/s]


In [80]:
print(dataset_dict)

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 4777
    })
    test: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 548
    })
})


In [81]:

train_df = dataset_dict['train'].to_pandas()
test_df = dataset_dict['test'].to_pandas()
train_df['partition'] = 'train'
test_df['partition'] = 'test'

combined_df = pd.concat([train_df, test_df])


combined_df.to_parquet('./data/codeNLU/h4ca20k_py/h4ca20k_py_.parquet')

In [83]:
dataset_dict.push_to_hub("graycatHCO3/CodeAlpaca-20K-Python")

Uploading the dataset shards: 100%|██████████| 1/1 [00:02<00:00,  2.14s/it]


CommitInfo(commit_url='https://huggingface.co/datasets/graycatHCO3/CodeAlpaca-20K-Python/commit/38048a8242c7b8745701cf970aa389c00308c5f5', commit_message='Upload dataset', commit_description='', oid='38048a8242c7b8745701cf970aa389c00308c5f5', pr_url=None, pr_revision=None, pr_num=None)

# 2. SFSA_e2t_0200 

In [1]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=open('./api/apikey/glmkeys.txt', 'r').read().strip())

In [2]:
import os 
import json
from src.utils.data_utils import format_data

In [3]:

input_folder = './data/SFSA_0200/sfsa_0200_01'
output_folder = './data/SFSA_0200/sfsa_0200_01_formatted'


In [10]:
standard_features = Features({
    'turn': Value('string'),
    'details': {
        'state_0': {
            'adversary_0(Thief)': [Value('float64')],
            'agent_0(Elf Wizard)': [Value('float64')],
            'agent_1(Human Warrior)': [Value('float64')],
        },
        'actions_0': {
            'adversary_0': Value('int64'),
            'agent_0': Value('int64'),
            'agent_1': Value('int64')
        },
        'reward_0': {
            'adversary_0': Value('float64'),
            'agent_0': Value('float64'),
            'agent_1': Value('float64')
        },
        'state_1': {
            'adversary_0(Thief)': [Value('float64')],
            'agent_0(Elf Wizard)': [Value('float64')],
            'agent_1(Human Warrior)': [Value('float64')]
        }
    }
})


In [11]:
sfsa_dict = {}
for file_name in os.listdir(input_folder):
    input_file_path = os.path.join(input_folder, file_name)

    with open(input_file_path, 'r') as file:
        data = json.load(file)

    formatted_data = format_data(data)
    dataset_name = file_name.replace('.json', '')
    dataset = Dataset.from_dict({'turn':[x['turn'] for x in formatted_data], 'details':[x['details'] for x in formatted_data]}, features=standard_features)
    sfsa_dict[dataset_name] = dataset
sfsa_dict = DatasetDict(sfsa_dict)
print(sfsa_dict)

DatasetDict({
    SFSA_0200_000001: Dataset({
        features: ['turn', 'details'],
        num_rows: 20
    })
    SFSA_0200_000002: Dataset({
        features: ['turn', 'details'],
        num_rows: 20
    })
    SFSA_0200_000003: Dataset({
        features: ['turn', 'details'],
        num_rows: 10
    })
    SFSA_0200_000004: Dataset({
        features: ['turn', 'details'],
        num_rows: 10
    })
    SFSA_0200_000005: Dataset({
        features: ['turn', 'details'],
        num_rows: 11
    })
    SFSA_0200_000006: Dataset({
        features: ['turn', 'details'],
        num_rows: 14
    })
    SFSA_0200_000007: Dataset({
        features: ['turn', 'details'],
        num_rows: 11
    })
    SFSA_0200_000008: Dataset({
        features: ['turn', 'details'],
        num_rows: 11
    })
    SFSA_0200_000009: Dataset({
        features: ['turn', 'details'],
        num_rows: 14
    })
    SFSA_0200_000010: Dataset({
        features: ['turn', 'details'],
        num_rows: 17
   

In [ ]:
sfsa_dict.push_to_hub("graycatHCO3/SFSA_e2t_0200")

In [ ]:
download_config = DownloadConfig(retries=5, use_auth_token=True)


In [6]:
sfsa = load_dataset("graycatHCO3/SFSA_e2t_0200",download_config=download_config)

SSLError: (MaxRetryError("HTTPSConnectionPool(host='huggingface.co', port=443): Max retries exceeded with url: /api/datasets/graycatHCO3/SFSA_e2t_0200/revision/df2ffbb221a51c7f81cc2d50f6e0d039df1a8917 (Caused by SSLError(SSLZeroReturnError(6, 'TLS/SSL connection has been closed (EOF) (_ssl.c:1129)')))"), '(Request ID: 4243dc56-a250-48c4-b5c9-fd30cd712185)')

In [14]:
sfsa_dict["SFSA_0200_000001"]

Dataset({
    features: ['turn', 'details'],
    num_rows: 20
})

In [ ]:

response = client.chat.completions.create(
    model="glm-4", 
    
    messages=[


        {"role": "system", "content": "The user will provide information from an episode in the reinforcement learning environment simple_adversary. The assistant needs to simulate appropriate dialogue between agents based on this information. Each line of dialogue should be prefixed with [0-2] to indicate the speaker, and suffixed with [0-3] to indicate the target. 0-3 represent adversary_0, agent_0, agent_1, and all, respectively. The dialogue should contain no more than five sentences."},
        {"role": "user", "content": sfsa_dict["SFSA_0200_000001"]["details"]},
        
    ],
)

print(response.choices[0].message.content)

In [ ]:
def dialogue(round):
    dialogue_dataset = Dataset.from_dict({})
    for i in round:

        response = client.chat.completions.create(
            model="glm-4", 
    
            messages=[


                {"role": "system", "content": "The user will provide information from an episode in the reinforcement learning environment simple_adversary. The assistant needs to simulate appropriate dialogue between agents based on this information. Each line of dialogue should be prefixed with [0-2] to indicate the speaker, and suffixed with [0-3] to indicate the target. 0-3 represent adversary_0, agent_0, agent_1, and all, respectively. The dialogue should contain no more than five sentences."},
                {"role": "user", "content": i["details"]},
        
            ],
        )

        if(response.choices[0].message.content == "1"):
            py_dataset.add_item(i)

    return py_dataset


In [ ]:
prompts  = h4ca20k["train"]

In [ ]:

features = Features({
    'prompt': Value('string'),
    'completion': Value('string')
})


py_dataset = Dataset.from_dict({'prompt':[], 'completion':[]},features=features)

In [ ]:

cot = 0
for i in prompts:

    response = client.chat.completions.create(
        model="glm-4", 
    
        messages=[


            {"role": "system", "content": "你需要判断用户接下来提供的内容是否和python有关。有关请回答1,无关请回答0"},
            {"role": "user", "content": i["prompt"]},
                
        
        ],
    )

    if (response.choices[0].message.content == "1"):
        
        
        if not(cot % 100):
            cot = cot+1
            print(cot)
            continue
        cot = cot+1

        py_dataset = concatenate_datasets([py_dataset,Dataset.from_dict({'prompt':[i['prompt']],'completion':[i['completion']]})])
    